### Simualting BlackJack

In [12]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import logging

In [13]:
# Logger
logFormatter = logging.Formatter('%(levelname)s:%(message)s')
logger = logging.getLogger('logger')
logger.setLevel(logging.DEBUG)
# create console handler for printing logging messages on the console
consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logFormatter)
logger.addHandler(consoleHandler)

In [14]:
class BlackJack_1P:
    
    
    def __init__(self, deck, ratios, cut_card, money):
        self.deck = deck
        self.ratios = ratios
        self.cut_card = cut_card
        self.money = money
        self.i = i
        self.result = list()
        self.ratios = {"blackjack":1.5, "win":1, "push":0, "lost": -1, "insurance": np.NaN} # ratios
    
    
    def ceil_to_10(self):
        func = lambda x: 10 if x > 10 else x
        return np.vectorize(func)

    
    def apply_basic_strategy(player, dealer, doubling_down=True):
        """
        Apply basic strategy rules given player and dealer hands.
        Details: http://www.cs.bu.edu/~hwxi/academic/courses/CS320/Spring02/assignments/06/basic-strategy.html
        :param player: player hand ceiled to 10
        :param dealer: dealer hand ceiled to 10
        :return action: one of hit, double, stand, split actions.
        """
        summe = sum(player)
        if player[0] != player[1]:
            if 1 not in player:
                if summe <= 8:
                    return "hit"
                elif summe == 9:
                    if dealer >= 3 and dealer <= 6:
                        return "double"
                    return "hit"
                elif summe == 10:
                    if dealer >= 2 and dealer <= 9:
                        return "double"
                    return "hit"
                elif summe == 11:
                    if dealer >= 2 and dealer <= 10:
                        return "double"
                    return "hit"
                elif summe == 12:
                    if dealer >= 4 and dealer <= 6:
                        return "stand"
                    return "hit"
                elif summe >= 13 and summe <= 16:
                    if dealer >= 2 and dealer <= 6:
                        return "stand"
                    return "hit"
                return "stand"
            else:
                if summe == 3 or summe == 4:
                    if dealer == 5 or dealer == 6:
                        return "double"
                    return "hit"
                elif summe == 5 or summe == 6:
                    if dealer >= 4 and dealer <= 6:
                        return "double"
                    return "hit"
                elif summe == 7:
                    if dealer >= 3 and dealer <= 6:
                        return "double"
                    return "hit"
                elif summe == 8:
                    if dealer >= 3 and dealer <= 6:
                        return "double"
                    elif dealer == 2 or dealer == 7 or dealer == 8:
                        return "stand"
                    return "hit"
                else:
                    return "stand"
        else:
            if doubling_down:
                if player[0] == 1:
                    return "split"
                elif player[0] == 2 or player[0] == 3:
                    if dealer >= 2 and dealer <= 7:
                        return "split"
                    return "hit"
                elif player[0] == 4:
                    if dealer == 5 or dealer == 6:
                        return "split"
                    return "hit"
                elif player[0] == 5:
                    if dealer >= 2 and dealer <= 9:
                        return "double"
                    return "hit"
                elif player[0] == 6 or player[0] == 7:
                    if dealer >= 2 and dealer <= player[0]:
                        return "split"
                    return "hit"
                elif player[0] == 8:
                    return "split"
                elif player[0] == 9:
                    if dealer == 1 or dealer == 7 or dealer == 10:
                        return "stand"
                    return "split"
                return "stand"
            else:
                if player[0] == 2 or player[0] == 3:
                    if dealer >= 4 and dealer <= 7:
                        return "split"
                    return "hit"
                elif player[0] == 4:
                    return "hit"
                elif player[0] == 6:
                    if dealer >= 3 and dealer <= 6:
                        return "split"
                    return "hit"

                
    def calculate_round(self, ratio, bet):
    
        earning = bet * ratio
        self.money = self.money + earning + np.sign(ratio+1) * bet
        return earning

    
    def play_round(self, round_, action, deck, player, dealer, bet, money):
        """
        Play one round of the game.
        :param round_: current round of the game
        :param action: action of the player. One of double, split, hit, stand.
        :param i: index of the deck
        :param cut_card: cut_card of the deck (where the game stops and a new one begins after shuffling)
        :param deck:
        :param bet: current bet of the player
        :money: current money of the player
        :return: i, cut_card, player, dealer, money
        """
        if action == "stand":
            dealer_sum = np.sum(dealer)
            player_sum = np.sum(player)
            if player_sum == 21:
                if dealer_sum == 21:
                    ratio = self.ratios["push"]
                    earning = calculate_round(ratio)
                    logger.debug("Round:{} Push! Player:{} Dealer:{} Current bet:{} Earning ratio:{} Earning:{} Current Money:{}".format(round_, player, dealer, bet, ratio, earning, self.money))
                else:
                    ratio = ratios["blackjack"]
                    earning = calculate_round(ratio)
                    logger.debug("Round:{} Player BlackJack! Player:{} Dealer:{}, Current bet:{} Earning ratio:{} Earning:{} Current Money:{}".format(round_, player, dealer, bet, ratio, earning, self.money))
            else:
                while dealer_sum < 17:
                    # win case
                    if dealer_sum > 21 or (21 - dealer_sum) > (21 - player_sum):
                        ratio = ratios["win"]
                        earning = calculate_round(ratio)
                        logger.debug("Round:{} Win! Player:{} Dealer:{}, Current bet:{} Earning ratio:{} Earning:{} Current Money:{}".format(round_, player, dealer, bet, ratio, earning, self.money))
                    # lost case
                    if dealer_sum == 21 or (21 - dealer_sum) < (21 - player_sum):
                        ratio = ratios["lost"]
                        earning = calculate_round(ratio)
                        logger.debug("Round:{} Lost! Player:{} Dealer:{}, Current bet:{} Earning ratio:{} Earning:{} Current Money:{}".format(round_, player, dealer, bet, ratio, earning, self.money))
                    dealer = np.append(dealer, self.ceil_to_10(self.deck[i]))
                    self.i += 1
                    self.cut_card -= 1
                    dealer_sum = np.sum(dealer)
        elif action == "hit":
            player = np.append(player, self.ceil_to_10(deck[i]))
            self.i += 1
            self.cut_card -= 1
            player_sum = np.sum(player)
            # busted case
            if player_sum > 21:
                ratio = ratios["lost"]
                earning = calculate_round(ratio)
                logger.debug("Round:{} Lost! Player:{} Dealer:{}, Current bet:{} Earning ratio:{} Earning:{} Current Money:{}".format(round_, player, dealer, bet, ratio, earning, self.money))
            else:
                action = self.apply_basic_strategy(player, dealer)
                play_round()

In [25]:
# parameters and fixies.
N = 4 # number of decks
money = 100 # entry money
bet = 1 # minimum bet
deck = np.repeat(np.arange(1,14), 4*N)
np.random.shuffle(deck) # inline shuffle
ratios = {"blackjack":1.5, "win":1, "push":0, "lost": -1, "insurance": np.NaN} # ratios
logger.info("Deck: {}".format(deck))
cut_card = np.random.randint(low=int(len(deck)/2), high=int(len(deck)))
logger.info("Deck length: {}".format(len(deck)))
logger.info("Cut card: {}".format(cut_card))
vfunc = np.vectorize(ceil_to_10)

INFO:Deck: [ 1  4  6  9  5  6  2 11 11  7  9  2  9  3 10  2  6  6 12  7 13 12  5 12  2
 11  5  5  3  9  8  4  7  1 13  2  2  3  5  4  9  6  7  8 13 11  2  9  9  9
 12  8 12  4  2  6  3 10  5  7  1 12  6 11  6 11 12  8  1 10  8  3  4  4  4
  7  6  6  7  8  4 11 11  9 12  9  8  1 11  5  4  5  5  7  6 10 10 11  1  3
 13 12  5  7  3  2 13 13 12 13  8  3  2 10 12 13 13 13 10  9  7  4  7 13  2
 11 10 11  3  1 11  8  5  3  4 10  4  7  8  3  7  5  2 11  1  2  9 13  1 11
 12  3  3  1 10  6  3  4 10  7  6  6  1 13  2  1 10  6 10  2 13  4  9  4 12
  5 10 12  1  1 12  5  4  3  9 10  7  8  8  6 10 13  1  8  9  8  7 11  5 13
 12  2  3  9  8  8  1  5]
INFO:Deck length: 208
INFO:Cut card: 164


In [5]:
# actions: double, split, hit, stand
# main loop
i = 0
round_ = 0
print("Cut card: {}".format(cut_card))
print("Deck length: {}".format(len(deck)))
while cut_card > 0 and i < len(deck):
    round_ += 1
    bet = 1
    money -= bet
    player = vfunc(deck[i:i+2])
    dealer = vfunc(deck[i+2:i+4])
    i = i+4
    cut_card -= 4
    # only at the beginning
    if np.sum(player) == 21:
        ratio = ratios["blackjack"]
        earning = bet * ratio
        money += earning + bet
        logger.debug("Round:{} Player BlackJack! Current bet:{} Earning ratio:{} Earning:{} Current Money:{}".format(round_, bet, ratio, earning, money))
        continue
    action = apply_basic_strategy(player, dealer[0])
    logger.debug("Round: {} Player: {} Dealer: {} Action: {}".format(round_, player, dealer, action))
    i, cut_card, player, dealer, money = play_game(round_, action, i, cut_card, deck, player, dealer, bet, money):

Cut card: 137
Deck length: 208
Round: 1 Player: [6 3] Dealer: 4 Action: double
Round: 2 Player: [9 2] Dealer: 10 Action: double
Round: 3 Player: [7 7] Dealer: 3 Action: split
Round: 4 Player: [ 8 10] Dealer: 8 Action: stand
Round: 5 Player: [3 2] Dealer: 8 Action: hit
Round: 6 Player: [4 7] Dealer: 10 Action: double
Round: 7 Player: [ 3 10] Dealer: 1 Action: hit
Round: 8 Player: [1 6] Dealer: 10 Action: hit
Round: 9 Player: [ 5 10] Dealer: 9 Action: hit
Round: 10 Player: [ 3 10] Dealer: 5 Action: stand
Round: 11 Player: [2 5] Dealer: 2 Action: hit
Round: 12 Player: [5 1] Dealer: 9 Action: hit
Round: 13 Player: [6 6] Dealer: 4 Action: split
Round: 14 Player: [5 1] Dealer: 2 Action: hit
Round: 15 Player: [6 4] Dealer: 10 Action: hit
Round: 16 Player: [10  9] Dealer: 7 Action: stand
Round: 17 Player: [ 2 10] Dealer: 4 Action: stand
Round: 18 Player: [4 3] Dealer: 3 Action: hit
Round: 19 Player: [ 1 10] Dealer: 2 Action: stand
Round: 20 Player: [7 8] Dealer: 4 Action: stand
Round: 21 Playe

In [10]:
abs(sum(player) - 21)

11

In [52]:
dealer = deck[2:4]
print(dealer)
dealer = np.append(dealer, deck[5])

[10  8]


In [53]:
dealer

array([10,  8,  8])

In [20]:
player = np.append(player, 1)

In [21]:
player

array([3, 7, 1])

In [23]:
sum(player)

11

In [29]:
np.heaviside(0,0)

AttributeError: module 'numpy' has no attribute 'heaviside'

In [33]:
np.sign(-1+1)

0

In [36]:
deck

array([ 1,  4,  6,  9,  5,  6,  2, 11, 11,  7,  9,  2,  9,  3, 10,  2,  6,
        6, 12,  7, 13, 12,  5, 12,  2, 11,  5,  5,  3,  9,  8,  4,  7,  1,
       13,  2,  2,  3,  5,  4,  9,  6,  7,  8, 13, 11,  2,  9,  9,  9, 12,
        8, 12,  4,  2,  6,  3, 10,  5,  7,  1, 12,  6, 11,  6, 11, 12,  8,
        1, 10,  8,  3,  4,  4,  4,  7,  6,  6,  7,  8,  4, 11, 11,  9, 12,
        9,  8,  1, 11,  5,  4,  5,  5,  7,  6, 10, 10, 11,  1,  3, 13, 12,
        5,  7,  3,  2, 13, 13, 12, 13,  8,  3,  2, 10, 12, 13, 13, 13, 10,
        9,  7,  4,  7, 13,  2, 11, 10, 11,  3,  1, 11,  8,  5,  3,  4, 10,
        4,  7,  8,  3,  7,  5,  2, 11,  1,  2,  9, 13,  1, 11, 12,  3,  3,
        1, 10,  6,  3,  4, 10,  7,  6,  6,  1, 13,  2,  1, 10,  6, 10,  2,
       13,  4,  9,  4, 12,  5, 10, 12,  1,  1, 12,  5,  4,  3,  9, 10,  7,
        8,  8,  6, 10, 13,  1,  8,  9,  8,  7, 11,  5, 13, 12,  2,  3,  9,
        8,  8,  1,  5])

In [38]:
deck[[0, 4]]

array([1, 5])